<a href="https://colab.research.google.com/github/charmerDark/quantum_svm_vqc_comparison/blob/main/hons_qsvm_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit

In [ ]:
import numpy as np

from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap,PauliFeatureMap
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.multiclass_extensions import AllPairs,OneAgainstRest
from qiskit.aqua.utils.dataset_helper import get_feature_dimension

import time

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report

In [ ]:
training_size=120
test_size=30
class_labels = [r'A', r'B', r'C']
data, target = datasets.load_iris(return_X_y=True)
sample_train, sample_test, label_train, label_test =train_test_split(data, target, test_size=30, random_state=42)
std_scale = StandardScaler().fit(sample_train)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)
samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)
training_input = {key: (sample_train[label_train == k, :])[:training_size]
                      for k, key in enumerate(class_labels)}
test_input = {key: (sample_test[label_test == k, :])[:test_size]
                  for k, key in enumerate(class_labels)}

In [ ]:
class circuit_result():
  '''
  class to store details of each circuit and later pickle it
  '''
  def details(self):
    print("Circuit type is: \t",self.circuit_type)
    print("feature_map used is :\t",self.feature_map)
    print("The paulis used were: \t",self.paulis)
    print("The entanglement layout is \t",self.entanglement)
    print("The repititions of feature map: \t",self.reps)
    print("running time is : \t",self.running_time)
    print("Accuracy report: \n",accuracy)
    print("Circuit Depth: \t", self.depth)
    print("Number of operations \t",self.count_ops)


  def __init__(self,circuit_type,feature_map,time,accuracy,depth,count_ops,paulis,reps,entanglement):
    self.circuit_type=circuit_type
    self.feature_map=feature_map
    self.running_time=time
    self.accuracy=accuracy
    self.depth=depth
    self.count_ops=count_ops
    self.paulis=paulis
    self.reps=reps
    self.entanglement=entanglement

In [ ]:
###main experiment loop
exp_count=1
experiments=[]
paulis=[["Z"],["X"],["Y"],["Z","ZZ"],["X","ZZ"],["Y","ZZ"]]
entanglements=["linear"]
for pauli in paulis:
  for entanglement in entanglements:
    for reps in range(1,2):
      backend = BasicAer.get_backend('qasm_simulator')
      feature_map = PauliFeatureMap(feature_dimension=get_feature_dimension(training_input),paulis=pauli,
                                reps=reps, entanglement=entanglement)
      svm = QSVM(feature_map, training_input, test_input,
                multiclass_extension=OneAgainstRest())
      quantum_instance = QuantumInstance(backend, shots=1024,
                                        seed_simulator=aqua_globals.random_seed,
                                        seed_transpiler=aqua_globals.random_seed)

      begin=time.time()
      result = svm.run(quantum_instance)
      end=time.time()
      print("Training took ", end - begin, " time" )
      ans=svm.predict(sample_test,quantum_instance=quantum_instance)
      accuracy=classification_report(ans,label_test)
      experiment=circuit_result("QSVM","PauliFeatureMap",end-begin,accuracy,svm.feature_map.depth(),svm.feature_map.count_ops(),pauli,reps,entanglement)
      experiment.details()
      experiments.append(experiment)

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/components/multiclass_extensions/multiclass_extension.py:34: DeprecationWarning: The package qiskit.aqua.components.multiclass_extensions is deprecated. For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.components.multiclass_extensions')
/usr/local/lib/python3.7/dist-packages/qiskit/aqua/algorithms/classifiers/qsvm/qsvm.py:104: DeprecationWarning: The package qiskit.aqua.algorithms.classifiers is deprecated. It was moved/refactored to qiskit_machine_learning.algorithms.classifiers (pip install qiskit-machine-learning). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  'qiskit-machine-learning')
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: The variable qiskit.aqua.aqua_globals is deprecated. It was moved/refactored to qiskit.utils.aqua_globals (pip install qiskit-terra).

Training took  241.43737649917603  time
Circuit tye is: 	 QSVM
feature_map used is :	 PauliFeatureMap
The paulis used were: 	 ['Z']
The entanglement layout is 	 linear
The repititions of feature map: 	 1
running time is : 	 241.43737649917603
Accuracy report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

Circuit Depth: 	 2
Number of operations 	 OrderedDict([('h', 4), ('p', 4)])
Training took  296.4930319786072  time


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Circuit tye is: 	 QSVM
feature_map used is :	 PauliFeatureMap
The paulis used were: 	 ['X']
The entanglement layout is 	 linear
The repititions of feature map: 	 1
running time is : 	 296.4930319786072
Accuracy report: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.30      0.46        30
           2       0.00      0.00      0.00         0

    accuracy                           0.30        30
   macro avg       0.33      0.10      0.15        30
weighted avg       1.00      0.30      0.46        30

Circuit Depth: 	 4
Number of operations 	 OrderedDict([('h', 12), ('p', 4)])
Training took  235.09958672523499  time
Circuit tye is: 	 QSVM
feature_map used is :	 PauliFeatureMap
The paulis used were: 	 ['Y']
The entanglement layout is 	 linear
The repititions of feature map: 	 1
running time is : 	 235.09958672523499
Accuracy report: 
               precision    recall  f1-score   support

    

In [ ]:
import pickle
with open("results","wb") as fil:
  pickle.dump(experiments,fil)
from google.colab import files
files.download('results')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>